In [50]:
import sys
import pandas as pd
import json
import os
import pickle
import re
import difflib
from difflib import SequenceMatcher
import jellyfish
pd.set_option('display.max_columns', 100)

In [51]:
import spotipy
import spotipy.util as util

In [52]:
user_name = "Pillsbury Darboy"
client_id = "bff1afed3b63465aa6cdba9ec40f6afd"
client_secret = "53634fc86e8a4f71b7459197084e3cdf"
redirect_uri = "http://localhost/"
scope = 'user-library-read'
from spotipy.util import prompt_for_user_token
token = prompt_for_user_token(username=user_name,scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri)
from spotipy import Spotify
spotify = Spotify(auth=token)

## Setlist Generator Class

In [121]:
class SetlistGenerator_Spotify(object):

    def __init__(self, seed_tracks='', seed_genres='', seed_artists='', setlist_length = 10):
        # build initial seed genres, tracks, and artists upon creation
        self.seed_genres = seed_genres
        self.seed_tracks = seed_tracks
        self.seed_artists = seed_artists
        self.setlist_length = setlist_length
        
        # necessary in order to query from the API
        self.AuthorizeSpotify()
        
        #Setlist starts with seed_track
        self.set_list = self.get_track_metadata_single()
        
        self.genres = ['chicago-house', 'club', 'dancehall', 'deep-house', 'detroit-techno', 'dub', 'edm',
         'electro', 'electronic', 'groove', 'house', 'idm', 'industrial', 'minimal-techno', 'progressive-house', 
          'soul', 'techno', 'trance']
        
        self.moods = ['happy', 'sad']
        
        self.track_pool = self.getRecommendations()
        
    def getRecommendations(self):
        # Query spotify for recommendations based on a track, genre, and/or artist
        recommendations = self.spotify.recommendations(seed_artists=self.seed_artists, 
                                                       seed_genres=self.seed_genres, 
                                                       seed_tracks=self.seed_tracks, 
                                                       limit = 50)
        # Convert spotify data into readable format
        metadata = self.get_track_metadata(recommendations)
        
        # Retrieve audio features for spotify data
        audio_features = pd.DataFrame(self.spotify.audio_features(self.get_track_ids(recommendations)))
        df = metadata.merge(audio_features)
        
        #Correct key + mode data 
        df = self.key_mode_convert(df)
        
        return(df)

    def getNextTrack(self, method = 0, current_song_key = None, min_dance=0.6):
        candidates = self.track_pool
        
        if current_song_key:
            pass
        else:
            current_song_key = int(self.set_list[-1:]['camelot'])
            
        if method == 0:
            print('Using method 0')
            candidates = self.filter_by_camelot(df = candidates, song_key_camelot = current_song_key)
            print(candidates['camelot'])
            candidates = self.filter_by_popularity(df = candidates)
            print(candidates['camelot'])
            candidates = self.filter_by_danceability(df = candidates, min_dance=min_dance)
            print(candidates['camelot'])
        else:
            pass
        
        self.set_list = self.set_list.append(candidates.sample(1))
        self.set_list = self.set_list.reset_index(drop=True)
        
        return(self.set_list)
    
    ############################################
    ######      SEED     UTILITIES        ######
    ############################################
    
    def changeSeedTracks(self, use_last = 2):
        self.seed_tracks = list(self.set_list['id'][-use_last:])
        
    def changeSeedGenres(self, new_genres):
        self.seed_genres = new_genres
        
    def changeSeedArtists(self, new_artists):
        self.seed_genres = artists
        
    ############################################
    ######    NARRATIVE     UTILITIES     ######
    ############################################
    
    def CreateTurnarounds(self, **kwargs):
        for key, value in kwargs.items():
            print(key, value)
        self.narrative_valence  = np.random()
        self.narrative_energy  = np.random()
        self.narrative_mode  = np.random()
        self.narrative_tempo  = np.random()
        self.narrative_popularity  = np.random()
        self.narrative_danceability  = np.random()
        self.narrative_genre = np.random()
        
    def CreateNarratives(self, valence=False, energy=False, mode = False, tempo = False,
                        popularity = False, danceability = False, genre = False):
        for key, value in kwargs.items():
            print(key, value)
        self.narrative_valence  = np.random()
        self.narrative_energy  = np.random()
        self.narrative_mode  = np.random()
        self.narrative_tempo  = np.random()
        self.narrative_popularity  = np.random()
        self.narrative_danceability  = np.random()
        self.narrative_genre = np.random()
        
    def NarrativeUtilDanceability(self):
        ## Mode 1 ##
        # Seed track is climax track.
        pass
    
    def NarrativeUtilMode(self):
        # Currently not implemented
        pass
    
    def NarrativeUtilPopularity(self):
        # Currently not implemented
        pass
    
    def NarrativeUtilEnergy(self):
        # Currently not implemented        
        pass
    
    def NarrativeUtilValence(self):
        # Currently not implemented
        pass
    
    ############################################
    ######     TRACK   POOL   UTILITIES   ######
    ############################################
        
    def getTrackPool(self):
        return self.track_pool
    
    def expandTrackPool(self):
        self.track_pool = self.track_pool.append(self.getRecommendations())
        self.cleanTrackPool()
        
        print('Expanded track pool')
    
    def cleanTrackPool(self):
        self.track_pool = self.track_pool[-self.track_pool['id'].isin(list(self.set_list['id']))]
        self.track_pool = self.track_pool.drop_duplicates()
        self.track_pool = self.track_pool.reset_index(drop=True)

        
    ############################################
    ######    TRACK SELECTION UTILITIES   ######
    ############################################
        
    def closest_keys(self, camelot):
        closest = []
        closest.append(camelot)
        closest.append((camelot + 1)%12)
        closest.append((camelot - 1)%12)
        return(closest)
    
    def FilterTracks(self, df, **kwargs):
        pass
        
    def filter_by_acousticness(self, df, min_acoustic = 0, max_acoustic = 1):
        df = df[df['acousticness'] >= min_acoustic]
        df = df[df['acousticness'] <= max_acoustic]
        return(df)
    
    def filter_by_camelot(self, df, song_key_camelot):
        filtered = []
        for i in self.closest_keys(song_key_camelot):
            filtered.append(pd.DataFrame(df[df['camelot'] == i]))
        return(pd.concat(filtered))
    
    def filter_by_danceability(self, df, min_dance = 0, max_dance = 1):

        df = df[df['danceability'] >= min_dance]
        df = df[df['danceability'] <= max_dance]
        return(df)
    
    def filter_by_energy(self, df, min_energy = 0, max_energy = 1):
        df = df[df['energy'] >= min_energy]
        df = df[df['energy'] <= max_energy]
        return(df)
    
    def filter_by_instrumentalness(self, df=None,
                                   min_instrumentalness = 0, 
                                   max_instrumentalness = 1):
        df = self.track_pool
        df = df[df['instrumentalness'] >= min_instrumentalness]
        df = df[df['instrumentalness'] <= max_instrumentalness]
        return(df)
    
    def filter_by_liveness(self, df=None, min_liveness = 0, max_liveness = 1):
        df = self.track_pool
        df = df[df['liveness'] >= min_liveness]
        df = df[df['liveness'] <= max_liveness]
        return(df)
    
    def filter_by_popularity(self, df, min_pop=0, max_pop=100):
        df = df[df['popularity'] >= min_pop]
        df = df[df['popularity'] <= max_pop]
        return(df)
    
    def filter_by_tempo(self, df, min_tempo = 90, max_tempo = 145):
        df = df[df['tempo'] >= min_tempo]
        df = df[df['tempo'] <= max_tempo]
        return(df)
    
    def filter_by_valence(self, df=None, min_valence = 0, max_valence = 1):
        df = self.track_pool
        df = df[df['valence'] >= min_valence]
        df = df[df['valence'] <= max_valence]
        return(df)
    

    ############################################
    ######        METADATA UTILITIES      ######
    ############################################
    
    def get_track_metadata(self, recommendations):
        tracklist_frames = []
        for track in recommendations['tracks']:
            tracklist_frames.append({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']})
        return(pd.DataFrame(tracklist_frames))

    def get_track_metadata_single(self):
        track_id = self.seed_tracks[0]
        track = self.spotify.track(track_id)
        metadata = pd.DataFrame({'track': track['name'],'artist': track['artists'][0]['name'],
                                     'duration':track['duration_ms'],
                                    'track_number': track['track_number'], 'id': track['id'], 
                                     'popularity':track['popularity'],
                                     'disc_number':track['disc_number'], 'album':track['album']['name']}, index =[0])
        audio_features = pd.DataFrame(self.spotify.audio_features(track_id))
        df = metadata.merge(audio_features)
        df = self.key_mode_convert(df)
        return(df)
    
    def get_track_ids(self, recommendations):
        id_list = []
        for spotify_id in recommendations['tracks']:
            id_list.append(spotify_id['id'])
        return(id_list)
    
    def key_mode_convert(self, recommendations):
        # TODO - Convert this method to one that uses key mapping
        # C = 0, Db = 1, D = 2, Eb = 3, E = 4, F = 5, F# = 6, G = 7, Ab = 8, A = 9, Bb = 10, B = 11
        df = recommendations
        camelot_list = []
        for i in range(0, len(df)):
            if df['mode'][i] == 1: #Major key
                if (df['key'][i] == 0):
                    camelot_list.append(7)
                if (df['key'][i] == 1):
                    camelot_list.append(2)
                if (df['key'][i] == 2):
                    camelot_list.append(9)
                if (df['key'][i] == 3):
                    camelot_list.append(4)
                if (df['key'][i] == 4):
                    camelot_list.append(11)                
                if (df['key'][i] == 5):
                    camelot_list.append(6)                
                if (df['key'][i] == 6):
                    camelot_list.append(1)                
                if (df['key'][i] == 7):
                    camelot_list.append(8)                
                if (df['key'][i] == 8):
                    camelot_list.append(3)                
                if (df['key'][i] == 9):
                    camelot_list.append(10)                
                if (df['key'][i] == 10):
                    camelot_list.append(5)
                if (df['key'][i] == 11):
                    camelot_list.append(0)
            else:               # Minor key
                if (df['key'][i] == 0):
                    camelot_list.append(4)
                if (df['key'][i] == 1):
                    camelot_list.append(11)
                if (df['key'][i] == 2):
                    camelot_list.append(6)
                if (df['key'][i] == 3):
                    camelot_list.append(1)
                if (df['key'][i] == 4):
                    camelot_list.append(8)                
                if (df['key'][i] == 5):
                    camelot_list.append(3)                
                if (df['key'][i] == 6):
                    camelot_list.append(10)                
                if (df['key'][i] == 7):
                    camelot_list.append(5)                
                if (df['key'][i] == 8):
                    camelot_list.append(0)                
                if (df['key'][i] == 9):
                    camelot_list.append(7)                
                if (df['key'][i] == 10):
                    camelot_list.append(2)
                if (df['key'][i] == 11):
                    camelot_list.append(9)
        df['camelot'] = camelot_list
        return(df)

    ############################################
    ######        SPOTIFY  UTILITIES      ######
    ############################################
    
    def SearchSpotify(self, search_string):
        return self.spotify.search(search_string)
    

    
    def AuthorizeSpotify(self):
        print('Authorizing Spotify')
        self.user_name = "Ronald Marp"
        self.client_id = '1e05e910ef664f27949005267fdd0936'
        self.client_secret ='7e65fb89a86a4a9c849924f5bd95661a'
        self.redirect_uri = "http://localhost/"
        self.scope = 'user-library-read'
        self.token = prompt_for_user_token(username=self.user_name,scope=self.scope,
                                           client_id=self.client_id,
                                           client_secret=self.client_secret,
                                           redirect_uri=self.redirect_uri)
        self.spotify = Spotify(auth=self.token)

## Additional Utilities

In [122]:
def search_for_artistID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]['artists'][0]
    print('Found artist %s' %result['name'])
    return(result['id'])

In [123]:
def search_for_trackID(search_string):
    result = spotify.search(str(search_string))['tracks']['items'][0]
    print('Found track %s by %s' %(result['name'], result['artists'][0]['name']))
    return(result['id'])

In [124]:
# Some example tracks to start with
## Percolator Meme - Teeth ['2QT5afoBT6nQabNuvcxXUa']

In [125]:
seed_artist_id = search_for_artistID('Vril Vortekz')
seed_track_id = search_for_trackID('Vril Vortekz')

Found artist Vril
Found track Vortekz by Vril


In [117]:
#seed_artist_id = search_for_artistID('Aleksi Perala')
#seed_track_id = search_for_trackID('Triadz Hubot')

In [126]:
d = SetlistGenerator_Spotify(seed_tracks=[seed_track_id], 
                             seed_artists=[seed_artist_id],
                             seed_genres=['minimal-techno', 'techno'])

Authorizing Spotify


In [127]:
d.expandTrackPool()

Expanded track pool


In [128]:
d.getNextTrack()

Using method 0
                                                album  \
9                           Bang the Acid - Rebanged!   
12                                                105   
40                                    Air conditionné   
47                          Amnesia Ibiza Summer 2012   
64  Berlin Afterhour 2 - From Minimal to Techno - ...   
85                                             People   
11                        Bounce Your Body To The Box   
13                                Frames Of Reference   
15                                  Clubtech Panorama   
27                                    Phantom Studies   
41                                  Future Modular EP   
50                                        Box Drop EP   
51        Marc Romboy Pres. Musica Electronica Vol. 4   
60                                         Hypnotized   
61                                      Play It Loud!   
66             DJ-Kicks (Daniel Avery) [Mixed Tracks]   
70              

,album,artist,disc_number,duration,id,popularity,track,track_number,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,camelot
0,Vortekz,Vril,1,320193,1w8cgxMCxfhrLkOEWGWWiJ,20,Vortekz,1,0.041100,https://api.spotify.com/v1/audio-analysis/1w8c...,0.823,320193,0.950,0.753,9,0.0683,-6.451,0,0.0941,123.469,4,https://api.spotify.com/v1/tracks/1w8cgxMCxfhr...,audio_features,spotify:track:1w8cgxMCxfhrLkOEWGWWiJ,0.465,7
1,Tidal/Moot Point - Single,Chymera,1,527073,4mabtOTo9WUIA1PoX8DBs4,4,Tidal,1,0.000899,https://api.spotify.com/v1/audio-analysis/4mab...,0.711,527074,0.572,0.891,7,0.0814,-9.669,1,0.0756,124.004,4,https://api.spotify.com/v1/tracks/4mabtOTo9WUI...,audio_features,spotify:track:4mabtOTo9WUIA1PoX8DBs4,0.674,8


In [119]:
while len(d.set_list) < 10:
    d.expandTrackPool()
    d.getNextTrack()
    d.changeSeedTracks()
    d.expandTrackPool()

d.set_list

Using method 0
Expanded track pool
Using method 0
Expanded track pool
Using method 0
Expanded track pool
Using method 0
Expanded track pool
Using method 0
Expanded track pool
Using method 0
Expanded track pool
Using method 0
Expanded track pool
Using method 0
Expanded track pool
Using method 0
Expanded track pool


,album,artist,disc_number,duration,id,popularity,track,track_number,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,camelot
0,Vortekz,Vril,1,320193,1w8cgxMCxfhrLkOEWGWWiJ,20,Vortekz,1,0.041100,https://api.spotify.com/v1/audio-analysis/1w8c...,0.823,320193,0.950,0.753,9,0.0683,-6.451,0,0.0941,123.469,4,https://api.spotify.com/v1/tracks/1w8cgxMCxfhr...,audio_features,spotify:track:1w8cgxMCxfhrLkOEWGWWiJ,0.4650,7
1,Warm Tonal Touch,Blawan,1,400781,3TEaKLoszXSdNmFxB8CVJS,7,Blue Bottle,4,0.001080,https://api.spotify.com/v1/audio-analysis/3TEa...,0.779,400782,0.980,0.834,2,0.1080,-5.602,1,0.1150,129.991,4,https://api.spotify.com/v1/tracks/3TEaKLoszXSd...,audio_features,spotify:track:3TEaKLoszXSdNmFxB8CVJS,0.0325,9
2,Space Ibiza,Oxia,1,433306,7bEPjQ1OKFUmg1m4oukyZG,2,Domino,32,0.002600,https://api.spotify.com/v1/audio-analysis/7bEP...,0.786,433307,0.687,0.912,1,0.2110,-10.235,0,0.0684,128.893,4,https://api.spotify.com/v1/tracks/7bEPjQ1OKFUm...,audio_features,spotify:track:7bEPjQ1OKFUmg1m4oukyZG,0.2140,11
3,Djax-Up-Beats - The Best Of - Volume 1,Mike Dearborn,1,377802,590yBscpz6L5cY6JKVkNoF,0,Birds On E,9,0.000971,https://api.spotify.com/v1/audio-analysis/590y...,0.612,377803,0.993,0.797,1,0.2950,-4.500,0,0.1240,138.961,4,https://api.spotify.com/v1/tracks/590yBscpz6L5...,audio_features,spotify:track:590yBscpz6L5cY6JKVkNoF,0.1830,11
4,Berlin Afterhour 2 - From Minimal to Techno - ...,Niklas Worgt,1,524232,1r1JF62uUs52MUxIVmxZPi,8,Berlin Bedlam,32,0.001700,https://api.spotify.com/v1/audio-analysis/1r1J...,0.868,524232,0.633,0.785,11,0.0828,-10.956,0,0.0900,124.997,4,https://api.spotify.com/v1/tracks/1r1JF62uUs52...,audio_features,spotify:track:1r1JF62uUs52MUxIVmxZPi,0.5410,9
5,Deep Touched - Deep & Tech House Collection #6,UGLH,1,503267,3MhrIn9vkPJxgf6w0FOaIE,0,Caracana - Club Mix,9,0.001910,https://api.spotify.com/v1/audio-analysis/3Mhr...,0.804,503267,0.775,0.914,11,0.1200,-8.564,0,0.0743,126.018,4,https://api.spotify.com/v1/tracks/3MhrIn9vkPJx...,audio_features,spotify:track:3MhrIn9vkPJxgf6w0FOaIE,0.4740,9
6,002,Ansome,1,337363,2Ahh0XB45YHaJ5Nkcj4p71,24,Tin,1,0.022300,https://api.spotify.com/v1/audio-analysis/2Ahh...,0.662,337363,0.978,0.811,1,0.0898,-8.844,0,0.0787,127.969,4,https://api.spotify.com/v1/tracks/2Ahh0XB45YHa...,audio_features,spotify:track:2Ahh0XB45YHaJ5Nkcj4p71,0.2800,11
7,Input-Output,Christian Smith,1,354497,67cz9RVm1yYoLEt9QJ6e5Y,30,Subzero - Album Mix,8,0.009380,https://api.spotify.com/v1/audio-analysis/67cz...,0.712,354498,0.890,0.923,10,0.1120,-8.736,0,0.0349,120.024,4,https://api.spotify.com/v1/tracks/67cz9RVm1yYo...,audio_features,spotify:track:67cz9RVm1yYoLEt9QJ6e5Y,0.0710,2
8,In Between,Marek Hemmann,1,359000,3UQ7GLdYhcWDCZSh1DPWoW,36,Yvette,7,0.104000,https://api.spotify.com/v1/audio-analysis/3UQ7...,0.723,359000,0.527,0.712,6,0.1320,-10.699,0,0.0846,128.047,4,https://api.spotify.com/v1/tracks/3UQ7GLdYhcWD...,audio_features,spotify:track:3UQ7GLdYhcWDCZSh1DPWoW,0.2640,10
9,Western Ways EP,Developer,1,309921,1PrBmN13Rw0NPSolkdBUlK,16,Western Ways - Pfirter Remix,4,0.000028,https://api.spotify.com/v1/audio-analysis/1PrB...,0.724,309921,0.916,0.889,6,0.1110,-8.453,1,0.0481,126.999,5,https://api.spotify.com/v1/tracks/1PrBmN13Rw0N...,audio_features,spotify:track:1PrBmN13Rw0NPSolkdBUlK,0.5560,1


In [103]:
d.filter_by_camelot(df=d.track_pool, song_key_camelot=3)

,album,artist,disc_number,duration,id,popularity,track,track_number,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,camelot
7,Black Hole,Boston 168,1,389420,52GRfo1fNyym5u2GXbOLtg,30,Terror Acid - Original Mix,3,0.001360,https://api.spotify.com/v1/audio-analysis/52GR...,0.680,389420,0.995,0.892,8,0.0917,-8.674,1,0.0513,127.983,4,https://api.spotify.com/v1/tracks/52GRfo1fNyym...,audio_features,spotify:track:52GRfo1fNyym5u2GXbOLtg,0.0416,3
29,EX (Performed Live At The Guggenheim NYC),Plastikman,1,476012,3G8o2J4Tiswq9F9aJCqxN4,9,EXpand,3,0.493000,https://api.spotify.com/v1/audio-analysis/3G8o...,0.765,476012,0.138,0.916,8,0.1070,-20.594,1,0.0476,121.995,4,https://api.spotify.com/v1/tracks/3G8o2J4Tiswq...,audio_features,spotify:track:3G8o2J4Tiswq9F9aJCqxN4,0.0331,3
34,Beyond The Valley,Anja Schneider,1,497173,0uT9JMRagU3OUj58K3ea2a,0,Maki,3,0.089400,https://api.spotify.com/v1/audio-analysis/0uT9...,0.801,497173,0.477,0.764,8,0.1120,-10.913,1,0.1930,129.217,4,https://api.spotify.com/v1/tracks/0uT9JMRagU3O...,audio_features,spotify:track:0uT9JMRagU3OUj58K3ea2a,0.5820,3
36,Turns,Barker & Baumecker,1,539034,7wUGO4eG3p5Ybe9F7SVgJ7,18,Noctural,6,0.003690,https://api.spotify.com/v1/audio-analysis/7wUG...,0.700,539035,0.916,0.859,8,0.0754,-8.300,1,0.0382,128.006,4,https://api.spotify.com/v1/tracks/7wUGO4eG3p5Y...,audio_features,spotify:track:7wUGO4eG3p5Ybe9F7SVgJ7,0.2610,3
63,Bohla,Blawan,1,330225,74PKtYK3H1CGztTCT2zswE,10,Bohla,1,0.022600,https://api.spotify.com/v1/audio-analysis/74PK...,0.838,330226,0.675,0.543,8,0.1790,-7.245,1,0.2730,130.138,4,https://api.spotify.com/v1/tracks/74PKtYK3H1CG...,audio_features,spotify:track:74PKtYK3H1CGztTCT2zswE,0.5330,3
68,Just Another Day,Carl Craig,1,579500,1Dm1h0x5pqBztbw10ZocVS,0,Sandstorms,3,0.006180,https://api.spotify.com/v1/audio-analysis/1Dm1...,0.650,579500,0.642,0.892,5,0.0951,-12.880,0,0.0364,120.001,4,https://api.spotify.com/v1/tracks/1Dm1h0x5pqBz...,audio_features,spotify:track:1Dm1h0x5pqBztbw10ZocVS,0.2770,3
104,Sunstep Ep,Oxia,1,482261,1Vvf0vr7dlKs5wyQeiXvW7,22,Whole Life - Original,2,0.014600,https://api.spotify.com/v1/audio-analysis/1Vvf...,0.807,482262,0.540,0.799,5,0.0714,-8.166,0,0.0814,124.996,4,https://api.spotify.com/v1/tracks/1Vvf0vr7dlKs...,audio_features,spotify:track:1Vvf0vr7dlKs5wyQeiXvW7,0.3270,3
106,The Rapture,Shlømo,1,409600,6RD8D6k0ZekJX2l7kknZn8,26,Burned Hand - Emmanuel Remix,4,0.200000,https://api.spotify.com/v1/audio-analysis/6RD8...,0.606,409600,0.906,0.877,8,0.2080,-9.381,1,0.0522,128.999,4,https://api.spotify.com/v1/tracks/6RD8D6k0ZekJ...,audio_features,spotify:track:6RD8D6k0ZekJX2l7kknZn8,0.0819,3
127,Gänseblümchen,Stimming,1,405931,0xtif1cPZHjjeGiyN6fmDc,35,Melodica - Original Mix,2,0.010900,https://api.spotify.com/v1/audio-analysis/0xti...,0.953,405932,0.451,0.929,8,0.1160,-10.264,1,0.1970,124.947,4,https://api.spotify.com/v1/tracks/0xtif1cPZHjj...,audio_features,spotify:track:0xtif1cPZHjjeGiyN6fmDc,0.5570,3
133,Baobab,Rodriguez Jr.,1,380818,7CtQHURo4pZPU2BLocdiQs,43,Heal Me,2,0.000779,https://api.spotify.com/v1/audio-analysis/7CtQ...,0.681,380819,0.553,0.611,5,0.3360,-12.027,0,0.0401,124.010,4,https://api.spotify.com/v1/tracks/7CtQHURo4pZP...,audio_features,spotify:track:7CtQHURo4pZPU2BLocdiQs,0.0888,3


In [78]:
def GenerateSetList(setlist_generator, setlength = 10, methods = 1):
    d = setlist_generator
    while len(d.set_list) <= setlength:
        d.getNextTrack()
        d.changeSeedTracks()
        d.expandTrackPool()
    return(d)
    

In [83]:
d.closest_keys(7)

[7, 8, 6]

In [80]:
d.set_list[-1:]['camelot']

9    2
Name: camelot, dtype: int64

In [ ]:
# Danceability Narrative Prototype
d = SetlistGenerator_Spotify(seed_tracks=[seed_track_id], seed_artists=[seed_artist_id], seed_genres=['techno'])
# First Expand the track pool to allow for more flexibility in narrative development
d.expandTrackPool()
d.expandTrackPool()
d.changeSeedGenres(['ambient', 'minimal-techno'])
d.expandTrackPool()
d.expandTrackPool()
d.changeSeedGenres(['ambient', 'techno'])
d.expandTrackPool()
d.expandTrackPool()
d.changeSeedGenres(['ambient'])
d.expandTrackPool()
d.expandTrackPool()
d.changeSeedGenres(['techno'])
d.CreateNarratives()
while len(d.set_list) < 10:
    d.getNextTrack()
    d.changeSeedTracks()
    d.expandTrackPool()

d.set_list

In [39]:
d.set_list['uri'].to_csv('../../../../spotify-downloader/output.txt', index=False)

In [34]:
spotify.recommendation_genre_seeds()

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


## Experimental Code
The code below is work in progress - it currently doesn't run and is left here until Version 1.0

In [ ]:
# Load and Process Datasets for big-data generation

spotify = pd.read_csv('data/spotify/spotify_database_mergedV2.csv')
del spotify['Unnamed: 0']
del spotify['Unnamed: 0.1']
spotify['artist_and_track'] = spotify['artist'] + ' ' + spotify['track']
print("Read and processed Spotify data")

discogs_m = pd.read_csv('data/discogs/discogs_mixed.csv')
discogs_r = pd.read_csv('data/discogs/discogs_releases.csv')
to_concat = [discogs_m, discogs_r]
discogs = pd.concat(to_concat)
del discogs_m
del discogs_r
del discogs['Unnamed: 0']
del discogs['genre']
del discogs['labels']
del discogs['length']
discogs['artist'] = discogs['artist'].apply(lambda x: re.sub(pattern='\([0-9]*\)', string=x, repl = '').strip())
discogs['artist_and_track'] = discogs['artist'] + ' ' + discogs['track']
print("Read and processed Discogs data")

mixesdb = pd.read_csv('data/mixesdb/mixesdb_all_sets_featurized.csv')
del mixesdb['Unnamed: 0']
mixesdb['artist_and_track'] = mixesdb['artist'] + ' ' + mixesdb['track']
print("Read and processed MixesDB data")
